In [2]:
import os
import glob #untuk mengambil semua file dalam satu folder
import requests #supaya bisa mengambil data dari website dengan cara request
import pandas as pd
import itertools #untuk menggabungkan dua list
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup #beautifulsoup untuk mengambil data dari website dengan cara parsing atau mengurai data html
from selenium import webdriver # webdriver bisa mengambil data yang dihasilkan oleh javascript


In [3]:
def scrapweb(var_list, awal_halaman, akhir_halaman):
    header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0'}
    halaman = awal_halaman
    while halaman <= akhir_halaman:
        url = f'https://www.lamudi.co.id/jakarta/house/buy/?page={halaman}'
        req = requests.get(url, headers=header, timeout=10)
        soup = BeautifulSoup(req.content, 'html.parser')
        semua_tempat = soup.find_all('div', class_='ListingCell-AllInfo ListingUnit')

        for i in semua_tempat:
            try:
                nama = i.find('h3', class_='ListingCell-KeyInfo-title')['title']
            except (AttributeError, KeyError):
                nama = None
            try:
                alamat = i.find('div', class_='ListingCell-KeyInfo-address ellipsis').find('span', class_='ListingCell-KeyInfo-address-text').text.strip()
            except (AttributeError, KeyError):
                alamat = None
            try:
                info_key = i.find('div', class_='KeyInformation_v2')
            
                # Mengambil semua div dengan class KeyInformation-attribute_v2
                attribute_divs = info_key.find_all('div', class_='KeyInformation-attribute_v2')
            
                # Mengambil nilai dari setiap atribut sesuai dengan labelnya
                for div in attribute_divs:
                    label = div.find('span', class_='KeyInformation-label_v2').text.strip()
                    value = div.find('span', class_='KeyInformation-value_v2').text.strip()
            
                    if label == 'Kamar tidur':
                        kamar_tidur = value
                    elif label == 'Bangunan':
                        luas_bangunan = value
                    elif label == 'Lahan':
                        luas_tanah = value
            
            except AttributeError:
                kamar_tidur = None
                luas_bangunan = None
                luas_tanah = None
            try:
                harga = i.find('div', class_='ListingCell-KeyInfo-PriceWrapper').find('span', class_='PriceSection-FirstPrice').text.strip()
            except (AttributeError, KeyError):
                harga = None
            try:
                deskripsi = i.find('div', class_='ListingCell-shortDescription').text.strip()
            except (AttributeError, KeyError):
                deskripsi = None
            var_list.append({
                'Nama': nama,
                'Alamat': alamat,
                'Kamar Tidur': kamar_tidur,
                'Luas Bangunan': luas_bangunan,
                'Luas Tanah': luas_tanah,
                'Harga': harga,
                'Deskripsi': deskripsi
            })
        halaman += 1

In [4]:
test = []
scrapweb(test, 1, 1)
len(test)

30

In [5]:
df = pd.DataFrame(test)
df

,Nama,Alamat,Kamar Tidur,Luas Bangunan,Luas Tanah,Harga,Deskripsi
0,Rumah Strategis Dijual 9 menit ke Stasiun Palm...,"Palmerah, Jakarta Barat",3,114 m²,57 m²,Rp1.500.000.000,"Jual cepat rumah minimalis seken, siap huni, b..."
1,ANA RUMAH BAGUS UK 4X8M DI JELAMBAR,"Jelambar, Jakarta Barat",3,60 m²,27 m²,Rp1.400.000.000,"DIJUAL RUMAH BAGUS UK 4X8M DI JELAMBAR LT 27M,..."
2,the Astelia Bintaro,"Bintaro, Jakarta Selatan",4,130 m²,60 m²,Rp1.980.000.000,Hunian Premium 3 Lantai Attic dan Outdoor Roof...
3,"DIJUAL Rumah Kedoya Sunrise Garden, TIDAK BANJ...","Kedoya, Jakarta Barat",5,230 m²,148 m²,Rp3.650.000.000,Dijual 2 Rumah Bebas banjir dijamin 2 lantai L...
4,Dijual Cepat Rumah 2 Lantai Di Rawa Buaya Ceng...,"Cengkareng, Jakarta Barat",4,85 m²,66 m²,Rp1.150.000.000,Jual Cepat Rumah Second 2 Lantai di Rawa Buaya...
5,Rumah Ter murah 2 lantai shm aman nyaman di M...,"Matraman, Jakarta Timur",2,58 m²,30 m²,Rp828.500.000,Di jual rumah murah dan elegant 2 lantai lingk...
6,Perumahan Oasis Ciracas Mansion Strategis sela...,"Tebet Timur, Jakarta Selatan",2,60 m²,60 m²,Rp960.000.000,Dijual Rumah Cluster Strategis Selangkah menuj...
7,Rumah murah jakarta 500 jutaan 10 unit lubang...,"Lubang Buaya, Jakarta Timur",2,36 m²,45 m²,Rp500.000.000,Dijual Cash dan tempo 10 unit lokasi jln luban...
8,Dijual Rumah Minimalis dengan SHM di Komplek K...,"Cengkareng, Jakarta Barat",3,120 m²,120 m²,Rp1.610.000.000,"Dijual rumah minimalis, bebas banjir, siap hun..."
9,Dijual Rumah trimaran permai PIK. uk 20x30.had...,"Pantai Indah Kapuk, Jakarta Utara",6,1300 m²,600 m²,Rp60.000.000.000,Dijual Rumah Trimaran Permai PIK1 Luas Tanah 6...


In [6]:
objek2 = []
scrapweb(objek2, 1, 50)
len(objek2)

1500

In [7]:
df3 = pd.DataFrame(objek2)
df3

,Nama,Alamat,Kamar Tidur,Luas Bangunan,Luas Tanah,Harga,Deskripsi
0,Rumah Strategis Dijual 9 menit ke Stasiun Palm...,"Palmerah, Jakarta Barat",3,114 m²,57 m²,Rp1.500.000.000,"Jual cepat rumah minimalis seken, siap huni, b..."
1,ANA RUMAH BAGUS UK 4X8M DI JELAMBAR,"Jelambar, Jakarta Barat",3,60 m²,27 m²,Rp1.400.000.000,"DIJUAL RUMAH BAGUS UK 4X8M DI JELAMBAR LT 27M,..."
2,the Astelia Bintaro,"Bintaro, Jakarta Selatan",4,130 m²,60 m²,Rp1.980.000.000,Hunian Premium 3 Lantai Attic dan Outdoor Roof...
3,"DIJUAL Rumah Kedoya Sunrise Garden, TIDAK BANJ...","Kedoya, Jakarta Barat",5,230 m²,148 m²,Rp3.650.000.000,Dijual 2 Rumah Bebas banjir dijamin 2 lantai L...
4,Dijual Cepat Rumah 2 Lantai Di Rawa Buaya Ceng...,"Cengkareng, Jakarta Barat",4,85 m²,66 m²,Rp1.150.000.000,Jual Cepat Rumah Second 2 Lantai di Rawa Buaya...
...,...,...,...,...,...,...,...
1495,hot listing jual cepat rumah kota wisata cibub...,"Cibubur, Jakarta Timur",3,100 m²,120 m²,Rp1.275.000.000,KOta wisata cibubur by sinar mas landdekat pin...
1496,Dijual rumah 1 lantai termurah di ciracas jaka...,"Ciracas, Jakarta Timur",2,60 m²,72 m²,Rp989.000.000,"TOWNHOUSE MEDINA MUSLIM JAKARTA ""Hunian Ideal ..."
1497,Rumah Florence PIK 10x20,Jakarta Utara,4,300 m²,200 m²,Rp8.800.000.000,DIJUAL RUMAH FLORENCE PIK UK 10x20 3 LANTAI L...
1498,DIJUAL RUMAH DESAIN MEGANIN 500 JUTAAN DI LEUW...,"Rawa Bunga, Jakarta Timur",2,40 m²,53 m²,Rp520.000.000,Hunian Minimalis Asri dan Sejuk Desain Rumah M...


In [ ]:
path = 'dataset/'

# Nama file CSV yang diinginkan
nama_file = 'jakarta.csv'

# Gabungkan path dan nama file
full_path = path + nama_file

# Simpan dataframe ke dalam file CSV
df3.to_csv(full_path, index=False)

print(f"Data berhasil disimpan ke: {full_path}")

In [6]:
test = []
scrapweb(test, 1, 1)

In [7]:
len(test)

20

In [8]:
objek1 = []
objek2 = []
objek3 = []
objek4 = []
objek5 = []
objek6 = []
objek7 = []
objek8 = []
objek9 = []
objek10 = []

In [9]:
scrapweb(objek1, 1, 20)

In [10]:
len(objek1)

20

In [11]:
df = pd.DataFrame(objek1)
df

,Nama Tempat,Harga,Alamat,Cicilan,Kamar Tidur,Kamar Mandi,Garasi,Luas Tanah,Luas Bangunan
0,Rumah 5 Menit Ke Tol Pasteur Minimalis Aman Ny...,Rp 975 Juta,"Cimahi Utara, Cimahi",Cicilan: 3 Jutaan per bulan,3,2,None,110 m²,86 m²
1,Cimahi Utara Home Industri Nusa Sari Rumah Bes...,"Rp 1,69 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,2,2,2,200 m²,180 m²
2,Dijual Cepat Dibulan Penuh Berkah Harga Nego R...,Rp 645 Juta,"Cimahi Selatan, Cimahi",Cicilan: 2 Jutaan per bulan,6,5,None,64 m²,128 m²
3,Rumah Asri 2 Lantai Dalam Komplek Strategis Di...,"Rp 2,6 Miliar","Cimahi Utara, Cimahi",Cicilan: 9 Jutaan per bulan,6,4,3,304 m²,250 m²
4,"Rumah Lelang Cimahi, Jawa Barat",Rp 385 Juta,"Cimahi Selatan, Cimahi",Cicilan: 1 Jutaan per bulan,1,1,None,72 m²,38 m²
5,Rumah 1.5 Lantai Permata Cimahi Butuh mendesak...,Rp 399 Juta,"Cimahi Utara, Cimahi",Cicilan: 1 Jutaan per bulan,2,1,1,60 m²,70 m²
6,Rumah di Jalan Permana Cimahi Utara 2 Lantai SHM,"Rp 1,7 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,3,2,1,171 m²,160 m²
7,Rumah Bagus 2 Lantai Komplek Taman Mutiara Cim...,"Rp 1,39 Miliar","Cimahi Tengah, Cimahi",Cicilan: 5 Jutaan per bulan,3,3,None,100 m²,170 m²
8,Sikat Hangatt Rumah di Taman Bumi Prima Cimahi...,"Rp 1,55 Miliar","Cimahi Utara, Cimahi",Cicilan: 5 Jutaan per bulan,3,2,1,203 m²,100 m²
9,Langka Gaada Lagii Rumah Di Taman Mutiara Cima...,"Rp 1,17 Miliar","Cimahi Tengah, Cimahi",Cicilan: 4 Jutaan per bulan,2,1,1,141 m²,70 m²


In [8]:
# Membuat DataFrame dari data_list
df = pd.DataFrame(test)

In [9]:
path = '/workspaces/codespaces-jupyter/cimahi/simpan/'
csv_path = os.path.join(path, 'Jual_rumah_cimahi_tes.csv')
df.to_csv(csv_path, index=False)  # index=False untuk menghilangkan index tambahan

# Memastikan header hanya ditambahkan sekali
df2 = pd.read_csv(csv_path)
df2

,Nama Tempat,Harga,Alamat,Cicilan,Kamar Tidur,Kamar Mandi,Garasi,Luas Tanah,Luas Bangunan
0,Cimahi Utara Home Industri Nusa Sari Rumah Bes...,"Rp 1,69 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,2,2,2.0,200 m²,180 m²
1,Dijual Cepat Dibulan Penuh Berkah Harga Nego R...,Rp 645 Juta,"Cimahi Selatan, Cimahi",Cicilan: 2 Jutaan per bulan,6,5,NaN,64 m²,128 m²
2,Rumah Asri 2 Lantai Dalam Komplek Strategis Di...,"Rp 2,6 Miliar","Cimahi Utara, Cimahi",Cicilan: 9 Jutaan per bulan,6,4,3.0,304 m²,250 m²
3,"Rumah Lelang Cimahi, Jawa Barat",Rp 385 Juta,"Cimahi Selatan, Cimahi",Cicilan: 1 Jutaan per bulan,1,1,NaN,72 m²,38 m²
4,Rumah Kost 3 Lantai di Kampung Ranca Cangkuang...,Rp 900 Juta,"Cimahi Selatan, Cimahi",Cicilan: 3 Jutaan per bulan,15,6,NaN,115 m²,90 m²
5,Rumah 1.5 Lantai Permata Cimahi Butuh mendesak...,Rp 399 Juta,"Cimahi Utara, Cimahi",Cicilan: 1 Jutaan per bulan,2,1,1.0,60 m²,70 m²
6,Rumah di Jalan Permana Cimahi Utara 2 Lantai SHM,"Rp 1,7 Miliar","Cimahi Utara, Cimahi",Cicilan: 6 Jutaan per bulan,3,2,1.0,171 m²,160 m²
7,Rumah 5 Menit Ke Tol Pasteur Minimalis Aman Ny...,Rp 975 Juta,"Cimahi Utara, Cimahi",Cicilan: 3 Jutaan per bulan,3,2,NaN,110 m²,86 m²
8,RUMAH CIPAGERAN CIMAHI CEPAT,Rp 650 Juta,"Cimahi Utara, Cimahi",Cicilan: 2 Jutaan per bulan,3,1,NaN,71 m²,72 m²
9,Murah! Dijual rumah minimalis dalam komplek Ci...,Rp 490 Juta,"Cimahi Utara, Cimahi",Cicilan: 1 Jutaan per bulan,2,1,1.0,72 m²,55 m²
